#### Carregamento de bibliotecas e DataSet

In [139]:
import pandas as pd
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

#Carrega o dataset
df = pd.read_csv(r'.\diabetes_dataset.csv', sep=',')
df_polars = pl.read_csv(r'.\diabetes_dataset.csv', separator=',')  

# Verifica se o dataframe foi carregado corretamente
print(df.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  target  
0 -0.002592  0.019907 -0.017646   151.0  
1 -0.039493 -0.068332 -0.092204    75.0  
2 -0.002592  0.002861 -0.025930   141.0  
3  0.034309  0.022688 -0.009362   206.0  
4 -0.002592 -0.031988 -0.046641   135.0  


#### Exercício 1 – Deteção de Outliers Pandas (IQR)

In [140]:
def detectar_outliers_iqr(df, cols):
    outliers_info = {}
    for col in cols:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
            outliers_info[col] = len(outliers)
            
    return outliers_info

outliers_ex1 = detectar_outliers_iqr(df, ['bmi', 'bp', 's5'])

print('Exercicio 1 - Número de outliers detectados:', outliers_ex1)

Exercicio 1 - Número de outliers detectados: {'bmi': 3, 'bp': 0, 's5': 4}


#### Exercício 2 – Capping de Valores Extremos Pandas

In [141]:
def aplicar_capping(df, cols):
    capped_df = df.copy()
    for col in cols:
        p5 = df[col].quantile(0.05)
        p95 = df[col].quantile(0.95)
        capped_df[col] = capped_df[col].clip(lower=p5, upper=p95)
    return capped_df

df_capped = aplicar_capping(df, ['bmi', 'bp', 's5'])

print('Exercicio 2 - Capping aplicado')

Exercicio 2 - Capping aplicado


#### Exercício 3 – Duplicados e Valores Nulos (simulados)

In [142]:
df3 = df.copy()
df3.loc[0, 'bmi'] = None
df3.loc[1, 'bp'] = None
df3 = pd.concat([df3, df3.iloc[[2]]]) 

duplicados_ex3 = df3.duplicated().sum()
print('Exercício 3 - Duplicados antes:', duplicados_ex3)

##remove duplicados
df3 = df3.drop_duplicates()
df3['bmi'].fillna(df3['bmi'].median(), inplace=True)
df3['bp'].fillna(df3['bp'].median(), inplace=True)

df_depois = df3.duplicated().sum()
print('Exercício 3 - Duplicados depois:', df_depois)

print('\nExercicio 3 - Nulos e duplicados removidos')

Exercício 3 - Duplicados antes: 1
Exercício 3 - Duplicados depois: 0

Exercicio 3 - Nulos e duplicados removidos


C:\Users\HP\AppData\Local\Temp\ipykernel_29452\503058056.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['bmi'].fillna(df3['bmi'].median(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_29452\503058056.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

#### Exercício 4 – Encoding de Variáveis Categóricas (simuladas)

In [143]:
df4 = df.copy()
df4['categoria_idade'] = pd.cut(df4['age'], bins=[0, 0.3, 0.6, 1.0], labels=['jovem', 'adulto', 'sénior'])
df4_label_encoded = df4.copy()
df4_label_encoded['categoria_idade'] = df4_label_encoded['categoria_idade'].astype('category').cat.codes
df4_onehot_encoded = pd.get_dummies(df4, columns=['categoria_idade'], drop_first=True)
print('Exercicio 4 - Codificação categórica concluída')


Exercicio 4 - Codificação categórica concluída


#### Exercício 5 – Escalonamento de Variáveis

In [144]:
df5 = df.copy()
scaler = StandardScaler()
df5_scaled = df5.copy()
df5_scaled[df5.columns] = scaler.fit_transform(df5[df5.columns])

print('Exercicio 5 - Escalonamento aplicado')

Exercicio 5 - Escalonamento aplicado


#### Exercício 6 – Feature Engineering + Seleção

In [145]:
df6 = df.copy()
df6['bmi_bp_interaction'] = df6['bmi'] * df6['bp']
#print(df6.head())

X = df6.drop(columns=['s6']) 
y = (df6['s6'] > df6['s6'].median()).astype(int)

X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
selector = SelectKBest(score_func=f_classif, k=5)
X_selected = selector.fit_transform(X_scaled, y)
selected_features = X.columns[selector.get_support()].tolist()

print('Exercício 6 - Features selecionadas:', selected_features)

Exercício 6 - Features selecionadas: ['age', 'bmi', 'bp', 's4', 's5']


## Polars

#### Exercício 1 – Deteção de Outliers Polars (IQR)

In [146]:
def detectar_outliers_iqr(df, cols):
    outliers_info = {}
    for col in cols:
        q1 = df.select(pl.col(col).quantile(0.25)).item()
        q3 = df.select(pl.col(col).quantile(0.75)).item()
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr
        outliers = df.filter((pl.col(col) < limite_inferior) | (pl.col(col) > limite_superior))
        outliers_info[col] = outliers.height
    return outliers_info

outliers_ex1 = detectar_outliers_iqr(df_polars, ['bmi', 'bp', 's5'])
print('Exercicio 1 - Outliers encontrados:', outliers_ex1)

Exercicio 1 - Outliers encontrados: {'bmi': 3, 'bp': 0, 's5': 4}


#### Exercício 2 – Capping de Valores Extremos Polars

In [147]:
def aplicar_capping(df, cols):
    df_capped = df.clone()
    for col in cols:
        p5 = df.select(pl.col(col).quantile(0.05)).item()
        p95 = df.select(pl.col(col).quantile(0.95)).item()
        df_capped = df_capped.with_columns(
            pl.col(col).clip(lower_bound=p5, upper_bound=p95).alias(col)
        )
    return df_capped

df_capped = aplicar_capping(df_polars, ['bmi', 'bp', 's5'])
print('Exercício 2 - Capping aplicado')

Exercício 2 - Capping aplicado


#### Exercício 3 – Duplicados e Valores Nulos (simulados) Polars

In [148]:
df3 = df_polars.clone()
df3 = df3.with_columns([
    pl.when(pl.arange(0, df3.height) == 0).then(None).otherwise(pl.col('bmi')).alias('bmi'),
    pl.when(pl.arange(0, df3.height) == 1).then(None).otherwise(pl.col('bp')).alias('bp')
])

df3 = df3.vstack(df3[2:3]) 

duplicados_ex3 = df3.is_duplicated().sum()
print('Exercício 3 - Duplicados antes:', duplicados_ex3)

# remove duplicados
df3 = df3.unique()
df3 = df3.with_columns([
    pl.col('bmi').fill_null(pl.col('bmi').median()),
    pl.col('bp').fill_null(pl.col('bp').median())
])
duplicados_depois = df3.is_duplicated().sum()
print('Exercício 3 - Duplicados depois:', duplicados_depois)

print('\nExercício 3 - Nulos imputados e duplicados removidos')

Exercício 3 - Duplicados antes: 2
Exercício 3 - Duplicados depois: 0

Exercício 3 - Nulos imputados e duplicados removidos


#### Exercício 4 – Encoding de Variáveis Categóricas (simuladas) Polars

In [149]:
df4 = df_polars.clone()
df4 = df4.with_columns([
    pl.when(pl.col('age') <= 0.3).then(pl.lit('jovem'))
    .when(pl.col('age') <= 0.6).then(pl.lit('adulto'))
    .otherwise(pl.lit('sénior'))
    .alias('categoria_idade')
])

# Label Encoding
df4_label_encoded = df4.with_columns([
    pl.col('categoria_idade').cast(pl.Categorical)
])

# One-Hot Encoding
df4_label_encoded = df4.to_dummies(columns=['categoria_idade'])
print('Exercício 4 - Codificação categórica concluída')

Exercício 4 - Codificação categórica concluída


#### Exercício 5 – Escalonamento de Variáveis Polars

In [150]:
df5 = df_polars.clone()
scaler = StandardScaler()

df5_scaled_np = scaler.fit_transform(df5.to_numpy())
df5_scaled = pl.DataFrame(df5_scaled_np, schema=df_polars.columns)
print(df5_scaled.head())
print('Exercício 5 - Escalonamento aplicado')

shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ age       ┆ sex       ┆ bmi       ┆ bp        ┆ … ┆ s4        ┆ s5        ┆ s6        ┆ target   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.8005    ┆ 1.065488  ┆ 1.297088  ┆ 0.459841  ┆ … ┆ -0.054499 ┆ 0.418531  ┆ -0.370989 ┆ -0.01471 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 9        │
│ -0.039567 ┆ -0.938537 ┆ -1.08218  ┆ -0.553505 ┆ … ┆ -0.830301 ┆ -1.436589 ┆ -1.938479 ┆ -1.00165 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 9        │
│ 1.793307  ┆ 1.065488  ┆ 0.934533  ┆ -0.119214 ┆ … ┆ -0.054499 ┆ 0.060156  

#### Exercício 6 – Feature Engineering + Seleção (Polars)

In [151]:
df6 = df_polars.clone()
df6 = df6.with_columns([
    (pl.col('bmi') * pl.col('bp')).alias('bmi_bp_interaction')
])

X = df6.drop('s6').to_pandas()
y = (df6['s6'].to_numpy() > df6['s6'].median()).astype(int)

X_scaled = StandardScaler().fit_transform(X)
selector = SelectKBest(score_func=f_classif, k=5)
X_selected = selector.fit_transform(X_scaled, y)
selected_features = X.columns[selector.get_support()].tolist()

print('Exercício 6 - Features selecionadas:', selected_features)

Exercício 6 - Features selecionadas: ['age', 'bmi', 'bp', 's4', 's5']


### Exercício 7 em PANDAS

In [152]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1 Carrega o dataset

DATA_PATH = r'.\train.csv'
df = pd.read_csv(DATA_PATH, sep=',')
#print(df.head())

# 2. Tratamento de valores nulos e duplicados
print('Duplicados antes:', df.duplicated().sum())
df = df.drop_duplicates()
print('Duplicados depois:', df.duplicated().sum())

df['Postal Code'].fillna(df['Postal Code'].median(), inplace=True)

# 3. Codificação de variáveis categóricas
cat_cols = df.select_dtypes(include='object').columns

drop_cols = ['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Customer Name',
'Product ID', 'Product Name']
df = df.drop(columns=drop_cols)

df_encoded = pd.get_dummies(df, columns=[col for col in cat_cols if col not in drop_cols], drop_first=True)

# 4. Escalonamento de variáveis numéricas
scaler = StandardScaler()
num_cols = df_encoded.select_dtypes(include='number').columns
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

# 5. Dataset pronto para modelos
print('\nDataset final pronto para modelos.')
print(df_encoded.head())


Duplicados antes: 0
Duplicados depois: 0

Dataset final pronto para modelos.
     Row ID  Postal Code     Sales  Ship Mode_Same Day  \
0 -1.731874    -0.401493  0.049776               False   
1 -1.731521    -0.401493  0.799801               False   
2 -1.731167     1.085497 -0.344944               False   
3 -1.730814    -0.685956  1.159887               False   
4 -1.730460    -0.685956 -0.332580               False   

   Ship Mode_Second Class  Ship Mode_Standard Class  Segment_Corporate  \
0                    True                     False              False   
1                    True                     False              False   
2                    True                     False               True   
3                   False                      True              False   
4                   False                      True              False   

   Segment_Home Office  City_Abilene  City_Akron  ...  Sub-Category_Envelopes  \
0                False         False       False

C:\Users\HP\AppData\Local\Temp\ipykernel_29452\2110225437.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Postal Code'].fillna(df['Postal Code'].median(), inplace=True)


### Exercício 7 em POLARS

In [153]:
import polars as pl
from sklearn.preprocessing import StandardScaler

# 1. Carregamento do dataset
DATA_PATH = r'.\train.csv'
df = pl.read_csv(DATA_PATH, separator=',')

# 2. Tratamento de valores nulos e duplicados
duplicados_antes = df.is_duplicated().sum()

print('Duplicados antes:', duplicados_antes)
df = df.unique()
print('Duplicados depois:', df.is_duplicated().sum())

postal_median = df.select(pl.col('Postal Code').median()).item()
df = df.with_columns(
            pl.col('Postal Code').fill_null(postal_median)
            )

# 3. Codificação de variáveis categóricas
drop_cols = ['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Customer Name','Product ID', 'Product Name']

df = df.drop(drop_cols)

cat_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype == pl.Utf8]

df = df.to_dummies(columns=cat_cols)

# 4. Escalonamento de variáveis numéricas
scaler = StandardScaler()
numeric_cols = [col for col, dtype in zip(df.columns, df.dtypes) if dtype in (pl.Int64, pl.Float64)]
df_scaled_np = scaler.fit_transform(df[numeric_cols].to_numpy())
df_scaled = df.with_columns([
pl.Series(name=col, values=df_scaled_np[:, i]) for i, col in enumerate(numeric_cols)
])

# 5. Dataset pronto para modelos
print('\n Dataset final pronto para modelos.')
print(df_scaled.head())

Duplicados antes: 0
Duplicados depois: 0

 Dataset final pronto para modelos.
shape: (5, 613)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Row ID    ┆ Ship Mode ┆ Ship      ┆ Ship Mode ┆ … ┆ Sub-Categ ┆ Sub-Categ ┆ Sub-Categ ┆ Sales    │
│ ---       ┆ _First    ┆ Mode_Same ┆ _Second   ┆   ┆ ory_Stora ┆ ory_Suppl ┆ ory_Table ┆ ---      │
│ f64       ┆ Class     ┆ Day       ┆ Class     ┆   ┆ ge        ┆ ies       ┆ s         ┆ f64      │
│           ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆          │
│           ┆ u8        ┆ u8        ┆ u8        ┆   ┆ u8        ┆ u8        ┆ u8        ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ -1.518019 ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ -0.29935 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 4       

## Desafio Autónomo

In [154]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

# 1. Carregamento do dataset
DATA_PATH = r'.\heart_failure_clinical_records_dataset.csv'
df_heart = pd.read_csv(DATA_PATH, sep=',', na_values=["N/D", "NA"])
#print(df_heart.head())

# A. Exploração inicial (EDA breve)
#Tratamento de valores nulos e duplicados, dimensoes do dataset
print('Duplicados antes:', df_heart.duplicated().sum())
df_heart = df_heart.drop_duplicates()
print('Duplicados depois:', df_heart.duplicated().sum())

print("\n Dimensões do dataset:")
print(df_heart.info())



Duplicados antes: 0
Duplicados depois: 0

 Dimensões do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtype

In [155]:
# B. Limpeza de outliers
# B. Limpeza de outliers
def remove_outliers(df, columns):
    outliers_count_before = {col: ((df[col] < df[col].quantile(0.25) - 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))).sum() +
                                    (df[col] > df[col].quantile(0.75) + 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))).sum()) for col in columns}
    
    for col in columns:
        IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
        lower_bound = df[col].quantile(0.25) - 1.5 * IQR
        upper_bound = df[col].quantile(0.75) + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    outliers_count_after = {col: ((df[col] < df[col].quantile(0.25) - 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))).sum() +
                                   (df[col] > df[col].quantile(0.75) + 1.5 * (df[col].quantile(0.75) - df[col].quantile(0.25))).sum()) for col in columns}
    
    return df, outliers_count_before, outliers_count_after

numeric_columns = df_heart.select_dtypes(include=[np.number]).columns
df_heart_cleaned, outliers_before, outliers_after = remove_outliers(df_heart, numeric_columns)

print("\nOutliers antes da limpeza:")
for col, count in outliers_before.items():
    print(f"{col}: {count} outliers")
    
print("\nOutliers depois da limpeza:")
for col, count in outliers_after.items():
    print(f"{col}: {count} outliers")



Outliers antes da limpeza:
age: 0 outliers
anaemia: 0 outliers
creatinine_phosphokinase: 29 outliers
diabetes: 0 outliers
ejection_fraction: 2 outliers
high_blood_pressure: 0 outliers
platelets: 21 outliers
serum_creatinine: 29 outliers
serum_sodium: 4 outliers
sex: 0 outliers
smoking: 0 outliers
time: 0 outliers
DEATH_EVENT: 0 outliers

Outliers depois da limpeza:
age: 0 outliers
anaemia: 0 outliers
creatinine_phosphokinase: 0 outliers
diabetes: 0 outliers
ejection_fraction: 0 outliers
high_blood_pressure: 0 outliers
platelets: 0 outliers
serum_creatinine: 3 outliers
serum_sodium: 0 outliers
sex: 0 outliers
smoking: 0 outliers
time: 0 outliers
DEATH_EVENT: 0 outliers


In [156]:
# C. Tratamento de valores em falta
    

# Imputação de valores em falta
# Preencher variáveis numéricas, usamos a mediana
for col in df_heart_cleaned.select_dtypes(include=[np.number]).columns:
    df_heart_cleaned[col].fillna(df_heart_cleaned[col].median(), inplace=True)
    

#Imputação de valores em falta para variáveis categóricas com moda
cat_cols = df_heart_cleaned.select_dtypes(include=[object]).columns
for col in cat_cols:
    moda = df_heart_cleaned[col].mode()[0]
    df_heart_cleaned[col].fillna(moda, inplace=True)

print('\nContagem de nulos')
print(df_heart_cleaned.isnull().sum()) #verifica se existem valores nulos
df_heart_cleaned = df_heart_cleaned.dropna() #remove os valores nulos
 



Contagem de nulos
age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64


C:\Users\HP\AppData\Local\Temp\ipykernel_29452\3475200205.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_heart_cleaned[col].fillna(df_heart_cleaned[col].median(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_29452\3475200205.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [157]:
#D. Engenharia de features
def categorizar_idade(idade):
    if idade < 40:
        return 'Menos de 40'
    elif 40 <= idade < 60:
        return '40-59'
    elif 60 <= idade < 80:
        return '60-79'
    else:
        return '80 ou mais'
    
#recurso a função
df_heart['categoria_idade'] = df_heart['age'].apply(categorizar_idade)
# recurso a numpy where
df_heart['categoria_sexo'] = np.where(df_heart['sex'] > 0, 'M', 'F')
# recurso a lambda
df_heart['categoria_serum'] = df_heart['serum_sodium'].apply(lambda x: 'Baixo' if x < 135 else 'Bom' if x < 145 else 'Elevado' )

print ('\nDistribuição de idades:')
print(df_heart['categoria_idade'].value_counts())
print ('\nDistribuição de Serum:')
print(df_heart['categoria_serum'].value_counts())
print ('\nDistribuição de Sexo:')
print(df_heart['categoria_sexo'].value_counts())
#print(df_heart.head())


Distribuição de idades:
categoria_idade
60-79         145
40-59         129
80 ou mais     25
Name: count, dtype: int64

Distribuição de Serum:
categoria_serum
Bom        205
Baixo       83
Elevado     11
Name: count, dtype: int64

Distribuição de Sexo:
categoria_sexo
M    194
F    105
Name: count, dtype: int64


In [158]:
#E. Codificação de variáveis categóricas

cat_cols = df_heart_cleaned.select_dtypes(include=[object]).columns

# Aplicar One-Hot Encoding 
df_one_hot = pd.get_dummies(df_heart_cleaned, columns=cat_cols, drop_first=True)

# Label Encoding para variáveis categóricas
label_encoder = LabelEncoder()  #Inicializando o LabelEncoder
df_label_encoded = df_heart_cleaned.copy()

for col in cat_cols:
    df_label_encoded[col] = label_encoder.fit_transform(df_label_encoded[col])

# Verificar resultados
print("One-Hot Encoding:", df_one_hot.head())
print("\nLabel Encoding:", df_label_encoded.head())

One-Hot Encoding:     age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
5  90.0        1                        47         0                 40   
6  75.0        1                       246         0                 15   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1   265000.0               1.9           130    1   
2                    0   162000.0               1.3           129    1   
3                    0   210000.0               1.9           137    1   
5                    1   204000.0               2.1           132    1   
6                    0   127000.0               1.2           137    1   

   smoking  time  DEATH_EVENT  
0        0     4            1  
2        1     7      

In [159]:
#F. Escalonamento / normalização

# Verificar tipos de dados antes da normalização
print(f'Tipos de dados antes da normalização:\n{df_heart_cleaned.dtypes}')


# Normalizar todas as variáveis numéricas
numeric_columns = df_heart_cleaned.select_dtypes(include=[np.number]).columns
scaler = StandardScaler() # Inicializando o StandardScaler
df_heart_cleaned[numeric_columns] = scaler.fit_transform(df_heart_cleaned[numeric_columns])

print(f'Tipos de dados depois da normalização:\n{df_heart_cleaned.dtypes}')


Tipos de dados antes da normalização:
age                         float64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object
Tipos de dados depois da normalização:
age                         float64
anaemia                     float64
creatinine_phosphokinase    float64
diabetes                    float64
ejection_fraction           float64
high_blood_pressure         float64
platelets                   float64
serum_creatinine            float64
serum_sodium                float64
sex                         float64
smoking                     float64
time                        float64
DEATH_EVE

In [160]:
#Explorar distribuição inicial
print("\nDistribuição inicial:")
num_cols =df_heart_cleaned.select_dtypes(include=[np.number]).columns
print(df_heart_cleaned[num_cols].describe().loc[['mean', 'min', 'max']])

#Calcular p5/p95 e aplicar capping
print("\nCapping:")
for col in num_cols:
    p5 = df_heart_cleaned[col].quantile(0.05)
    p95 = df_heart_cleaned[col].quantile(0.95)
    df_heart_cleaned[col + "_raw"] = df_heart_cleaned[col] #guardar os valores originais
    print(f"Coluna: {col} | P5: {p5:.2f} | P95: {p95:.2f}")
    df_heart_cleaned[col] = df_heart_cleaned[col].clip(p5, p95)   
    #print(f"Coluna: {col} | Média: {df_heart_cleaned[col].mean():.2f} | Mínimo: {df_heart_cleaned[col].min():.2f} | Máximo: {df_heart_cleaned[col].max():.2f}")

#Avaliar impacto do capping
before = df_heart_cleaned[[col + "_raw" for col in num_cols]].describe().loc[['mean', 'std']]
after = df_heart_cleaned[num_cols].describe().loc[['mean', 'std']]
print("\nImpacto do capping:")
print(before.join(after, lsuffix='_before', rsuffix='_after'))    


Distribuição inicial:
               age       anaemia  creatinine_phosphokinase      diabetes  \
mean  2.220446e-16  5.749369e-17              4.758099e-17 -6.344132e-17   
min  -1.745855e+00 -9.393364e-01             -1.100559e+00 -8.503393e-01   
max   2.868475e+00  1.064581e+00              3.101718e+00  1.176001e+00   

      ejection_fraction  high_blood_pressure     platelets  serum_creatinine  \
mean      -1.804112e-16         4.758099e-17 -4.807662e-16      7.930164e-17   
min       -2.069263e+00        -7.819916e-01 -1.997878e+00     -1.616121e+00   
max        2.295445e+00         1.278786e+00  2.554110e+00      2.970169e+00   

      serum_sodium           sex       smoking          time   DEATH_EVENT  
mean -1.056694e-15 -8.326673e-17 -1.586033e-17 -1.903239e-16  6.344132e-17  
min  -3.171117e+00 -1.341641e+00 -6.882472e-01 -1.683733e+00 -6.117460e-01  
max   2.884329e+00  7.453560e-01  1.452966e+00  1.993861e+00  1.634665e+00  

Capping:
Coluna: age | P5: -1.58 | P95: 1.

In [161]:
#G. Validação final

# Verificar duplicados
duplicates = df_heart_cleaned.duplicated().sum()
print(f'Número de duplicados: {duplicates}')

# Verificar ausência de valores nulos
print(f'\nValores ausentes finais:\n{df_heart_cleaned.isnull().sum()}')

# Verificar colunas numéricas escalonadas
print(f'\nColunas numéricas escalonadas:\n{df_heart_cleaned[numeric_columns].head()}')

# Verificar colunas categóricas codificadas (One-Hot ou Label Encoding)
print(f'\nColunas categóricas codificadas:\n{df_heart_cleaned.select_dtypes(include=[object]).head()}')


Número de duplicados: 0

Valores ausentes finais:
age                             0
anaemia                         0
creatinine_phosphokinase        0
diabetes                        0
ejection_fraction               0
high_blood_pressure             0
platelets                       0
serum_creatinine                0
serum_sodium                    0
sex                             0
smoking                         0
time                            0
DEATH_EVENT                     0
age_raw                         0
anaemia_raw                     0
creatinine_phosphokinase_raw    0
diabetes_raw                    0
ejection_fraction_raw           0
high_blood_pressure_raw         0
platelets_raw                   0
serum_creatinine_raw            0
serum_sodium_raw                0
sex_raw                         0
smoking_raw                     0
time_raw                        0
DEATH_EVENT_raw                 0
dtype: int64

Colunas numéricas escalonadas:
        age   anaemia

In [162]:
# H. Exportação

# Exportar para Pandas
DATA_PATH_PANDAS = r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\Avaliacao\heart_prepared_pandas.csv'

df_heart_cleaned.to_csv(DATA_PATH_PANDAS, index=False)

# Exportar para Polars
import polars as pl

DATA_PATH_POLARS = r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\Avaliacao\heart_prepared_polars.csv'

df_polars = pl.from_pandas(df_heart_cleaned)
df_polars.write_csv(DATA_PATH_POLARS)